In [1]:
!git clone https://github.com/afrenkai/DS-3010-Final.git

Cloning into 'DS-3010-Final'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 74 (delta 25), reused 62 (delta 17), pack-reused 0 (from 0)
Receiving objects: 100% (74/74), 16.14 MiB | 6.06 MiB/s, done.
Resolving deltas: 100% (25/25), done.


In [13]:
%cd DS-3010-Final

/content/DS-3010-Final


In [14]:
!ls

cv.py  data.ipynb  main.py    requirements.txt	sgemm_product.csv
Data   LICENSE	   README.md  setup.bat


In [15]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == 

In [35]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [54]:
train_df = pd.read_csv('Data/SGEMM_train.csv')

# preprocessing

In [55]:
cols_to_combine = ['Run1 (ms)', 'Run2 (ms)', 'Run3 (ms)', 'Run4 (ms)']

In [56]:
train_df['DELTA_RUNTIME'] = train_df.apply(
    lambda row: np.mean([row['Run1 (ms)'], row['Run2 (ms)'], row['Run3 (ms)'], row['Run4 (ms)']]),
    axis=1
)


In [57]:
for col in train_df.columns:
  if col in cols_to_combine:
    train_df = train_df.drop(col, axis = 1)

In [58]:
train_df

,MWG,NWG,KWG,MDIMC,NDIMC,MDIMA,NDIMB,KWI,VWM,VWN,STRM,STRN,SA,SB,DELTA_RUNTIME
0,128,128,32,16,16,8,8,2,4,8,0,1,1,0,224.4225
1,32,64,32,16,8,8,32,8,2,1,0,0,0,1,45.5625
2,64,32,16,8,8,8,8,2,1,2,1,1,1,1,37.8800
3,64,128,32,16,32,16,16,2,4,1,0,0,1,1,41.1375
4,128,128,16,8,32,32,32,8,2,1,0,0,1,1,111.1725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193275,128,16,32,16,16,32,16,2,4,1,1,1,1,1,60.8325
193276,64,128,16,16,8,32,32,8,1,1,1,1,0,1,110.3775
193277,128,64,16,8,16,16,16,2,1,2,0,0,1,1,172.1650
193278,64,32,32,16,16,32,8,8,2,2,0,1,1,0,33.5800


In [48]:
def norm(x, xmin, xmax, a, b):
  '''
  Restricts x values to range of [xmin, xmax]
  '''
  numerator = x - xmin
  denominator = xmax - xmin
  return (numerator / denominator) * (b - a) + a


In [50]:
# min = 0
# max = 1
# train_df = train_df.apply(
#     lambda row: (norm(row, row.min(), row.max(), min, max))
# )

In [51]:
train_df

,MWG,NWG,KWG,MDIMC,NDIMC,MDIMA,NDIMB,KWI,VWM,VWN,STRM,STRN,SA,SB,DELTA_RUNTIME
0,1.000000,1.000000,1.0,0.333333,0.333333,0.000000,0.000000,0.0,0.428571,1.000000,0.0,1.0,1.0,0.0,0.063468
1,0.142857,0.428571,1.0,0.333333,0.000000,0.000000,1.000000,1.0,0.142857,0.000000,0.0,0.0,0.0,1.0,0.009694
2,0.428571,0.142857,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.142857,1.0,1.0,1.0,1.0,0.007385
3,0.428571,1.000000,1.0,0.333333,1.000000,0.333333,0.333333,0.0,0.428571,0.000000,0.0,0.0,1.0,1.0,0.008364
4,1.000000,1.000000,0.0,0.000000,1.000000,1.000000,1.000000,1.0,0.142857,0.000000,0.0,0.0,1.0,1.0,0.029420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193275,1.000000,0.000000,1.0,0.333333,0.333333,1.000000,0.333333,0.0,0.428571,0.000000,1.0,1.0,1.0,1.0,0.014285
193276,0.428571,1.000000,0.0,0.333333,0.000000,1.000000,1.000000,1.0,0.000000,0.000000,1.0,1.0,0.0,1.0,0.029181
193277,1.000000,0.428571,0.0,0.000000,0.333333,0.333333,0.333333,0.0,0.000000,0.142857,0.0,0.0,1.0,1.0,0.047757
193278,0.428571,0.142857,1.0,0.333333,0.333333,1.000000,0.000000,1.0,0.142857,0.142857,0.0,1.0,1.0,0.0,0.006092
